In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [3]:
from we import get_we, initiate_model

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from scipy.stats import f_oneway

In [6]:
from sklearn.metrics import mean_absolute_error, accuracy_score

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [9]:
labels = [m['label'] for m in models]

In [10]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [11]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.909603,0.551025,0.436937,0.789782,0.311517,0.417476,0.629341,0.913722,0.349834,0.838988,...,0.285154,0.057887,0.030874,0.310204,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.901779,0.549880,0.427826,0.797102,0.313900,0.419226,0.623478,0.916539,0.355807,0.844714,...,0.287610,0.054251,0.018604,0.305226,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.899570,0.555891,0.418186,0.805754,0.319232,0.412976,0.618457,0.914183,0.367882,0.845985,...,0.287883,0.057476,0.013614,0.290463,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.898879,0.556306,0.417821,0.805256,0.318463,0.412505,0.618276,0.914983,0.367654,0.846577,...,0.288183,0.057827,0.014050,0.290994,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.895888,0.554461,0.420659,0.799339,0.322441,0.418141,0.618820,0.921033,0.367005,0.845904,...,0.293985,0.057312,0.012175,0.298473,singular,feminine,abbaye,NOUN,NaN,NaN


In [12]:
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'POS'

# Adjectives

In [13]:
pos = ['ADJ']

Split each model into train and test:

In [14]:
X_adj_train = []
y_adj_train = []

X_adj_test = []
y_adj_test = []

In [15]:
for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='ADJ',
                                           split=True,
                                           balance=True)
    X_adj_train.append(xtr)
    X_adj_test.append(xtst)
    
    y_adj_train.append(ytr)
    y_adj_test.append(ytst)

In [16]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['Train size'] = [len(x) for x in X_adj_train]
dataset_sizes['Test size'] = [len(x) for x in X_adj_test]
dataset_sizes

,Train size,Test size
flau_small_c,3011,753
flau_base_u,3585,897
flau_base_c,3011,753
flau_large_c,3011,753
cam_base,1668,418
xlm_large,324,82
xlm_base,324,82
bert_base_u,662,166
distilbert_base,420,106
bert_base_c,420,106


## Non-independent dims

ANOVA test with p-value < 0.01:

In [17]:
anova_dims = [get_anova_dims(X_adj_train[i], y_adj_train[i]) for i in range(len(models))]

In [18]:
mi_dims = [get_mi_dims(X_adj_train[i], y_adj_train[i]) for i in range(len(models))]

In [19]:
non_ind_dims = [list(set(anova_dims[i]).intersection(mi_dims[i])) for i in range(len(models))]

Stats about the number of dimensions for each model:

In [20]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [21]:
non_ind_df['ANOVA'] = [len(x) for x in anova_dims]
non_ind_df['MI'] = [len(x) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x) for x in non_ind_dims]

In [22]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,379,369,303
flau_base_u,404,651,361
flau_base_c,324,449,196
flau_large_c,712,757,593
cam_base,254,577,202
xlm_large,10,480,7
xlm_base,90,441,51
bert_base_u,196,454,123
distilbert_base,182,466,134
bert_base_c,6,458,5


In [23]:
dims = {}

In [24]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = X_adj_train[i].columns
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [25]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [26]:
lr_res = [lr(X_adj_train[i], y_adj_train[i]) for i in range(len(models))]

Train Perceptron 10 times and get average weights:

In [27]:
perceptron_res = [perceptron(X_adj_train[i], y_adj_train[i]) for i in range(len(models))]

Compute correlation to the gender vector:

In [28]:
corr_res = [correlation(X_adj_train[i], y_adj_train[i]) for i in range(len(models))]

In [29]:
for i in range(len(models)):
    for alpha in alphas:
        num_imp_dims = len(X_adj_test[i].columns)*alpha//100
        lr_dims = [str(x[0]) for x in lr_res[i][:num_imp_dims]]
        perc_dims = [str(x[0]) for x in perceptron_res[i][:num_imp_dims]]
        corr_dims = [str(x[0]) for x in corr_res[i][:num_imp_dims]]
        dims[labels[i]][f'LR{alpha}'] = lr_dims
        dims[labels[i]][f'Perc{alpha}'] = perc_dims
        dims[labels[i]][f'Corr{alpha}'] = corr_dims
        dims[labels[i]][f'All imp dims{alpha}'] = list(set(lr_dims).intersection(perc_dims).intersection(corr_dims))

## Compute medians

In [30]:
medians = {}

In [31]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = {}
        dim_list = dims[labels[i]][dim_group]
        # Median of dimensions where feature vector is equal to 0
        median_0 = X_adj_train[i][y_adj_train[i] == 0][dim_list].median()
        # Median of dimensions where feature vector is equal to 1
        median_1 = X_adj_train[i][y_adj_train[i] == 1][dim_list].median()
        
        medians[labels[i]][dim_group]['0'] = median_0
        medians[labels[i]][dim_group]['1'] = median_1


In [32]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group])

In [33]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,512,768,768,1024,768,1024,768,768,768,768
ANOVA,379,404,324,712,254,10,90,196,182,6
MI,369,651,449,757,577,480,441,454,466,458
All non ind,303,361,196,593,202,7,51,123,134,5
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,0,0,0,1,0,1,0,0,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [34]:
y_preds = {}

In [35]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        dim_list = dims[labels[i]][dim_group]
        mae0 = X_adj_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['0'], x), axis=1)
        mae1 = X_adj_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['1'], x), axis=1)
        # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
        y_preds[labels[i]][dim_group] = (mae0 > mae1).apply(int)
    

## Compute accuracies

In [36]:
accs = {}

In [37]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_true = y_adj_test[i]
        y_pred = y_preds[labels[i]][dim_group]
        if any(y_pred):
            acc = accuracy_score(y_true, y_pred)
        else:
            acc = 0
        accs[labels[i]][dim_group] = acc

In [38]:
accs_df = pd.DataFrame(accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.888446,0.576366,0.571049,0.868526,0.543062,0.585366,0.548780,0.512048,0.613208,0.547170
ANOVA,0.891102,0.586399,0.573705,0.867198,0.543062,0.646341,0.621951,0.512048,0.594340,0.594340
MI,0.891102,0.576366,0.580345,0.872510,0.543062,0.548780,0.548780,0.512048,0.613208,0.556604
All non ind,0.893758,0.590858,0.575033,0.868526,0.543062,0.585366,0.634146,0.512048,0.603774,0.584906
LR1,0.709163,0.530658,0.548473,0.738380,0.543062,0.487805,0.536585,0.500000,0.556604,0.547170
Perc1,0.770252,0.583055,0.601594,0.788845,0.552632,0.573171,0.560976,0.518072,0.622642,0.528302
Corr1,0.758300,0.610925,0.579017,0.802125,0.576555,0.646341,0.585366,0.493976,0.613208,0.594340
All imp dims1,0.000000,0.000000,0.000000,0.698539,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
LR5,0.808765,0.584169,0.590969,0.814077,0.528708,0.585366,0.597561,0.512048,0.632075,0.537736
Perc5,0.852590,0.622074,0.618858,0.871182,0.550239,0.512195,0.560976,0.512048,0.622642,0.556604


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [39]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,0.002656,0.010033,0.002656,-0.001328,0.000000,0.060976,0.073171,0.000000,-0.018868,0.047170
MI,0.002656,0.000000,0.009296,0.003984,0.000000,-0.036585,0.000000,0.000000,0.000000,0.009434
All non ind,0.005312,0.014493,0.003984,0.000000,0.000000,0.000000,0.085366,0.000000,-0.009434,0.037736
LR1,-0.179283,-0.045708,-0.022576,-0.130146,0.000000,-0.097561,-0.012195,-0.012048,-0.056604,0.000000
Perc1,-0.118194,0.006689,0.030544,-0.079681,0.009569,-0.012195,0.012195,0.006024,0.009434,-0.018868
Corr1,-0.130146,0.034560,0.007968,-0.066401,0.033493,0.060976,0.036585,-0.018072,0.000000,0.047170
All imp dims1,-0.888446,-0.576366,-0.571049,-0.169987,-0.543062,-0.085366,-0.548780,-0.512048,-0.613208,-0.547170
LR5,-0.079681,0.007804,0.019920,-0.054449,-0.014354,0.000000,0.048780,0.000000,0.018868,-0.009434
Perc5,-0.035857,0.045708,0.047809,0.002656,0.007177,-0.073171,0.012195,0.000000,0.009434,0.009434


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [40]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c Perc25 0.01859229747675961
flau_base_u Perc10 0.055741360089186176
flau_base_c Perc10 0.07038512616201853
flau_large_c Perc10 0.02257636122177953
cam_base Corr1 0.03349282296650713
xlm_large ANOVA 0.060975609756097615
xlm_base All non ind 0.08536585365853655
bert_base_u Perc1 0.0060240963855421326
distilbert_base LR5 0.018867924528301883
bert_base_c ANOVA 0.047169811320754707


In [41]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c Perc25 0.9070385126162018
flau_base_u Perc10 0.6321070234113713
flau_base_c Perc10 0.6414342629482072
flau_large_c Perc10 0.8911022576361222
cam_base Corr1 0.5765550239234449
xlm_large ANOVA 0.6463414634146342
xlm_base All non ind 0.6341463414634146
bert_base_u Perc1 0.5180722891566265
distilbert_base LR5 0.6320754716981132
bert_base_c ANOVA 0.5943396226415094


In [42]:
if 'pos_adj.pickle' not in os.listdir('../Data/best_results/'):
    best_res = {}
    for label in labels:
        best_res[label] = {}
        best_res[label]['best_dim_set'] = []
        best_res[label]['best_dims'] = []
        best_res[label]['accs'] = []
        best_res[label]['gains'] = []
        best_res[label]['medians_0'] = []
        best_res[label]['medians_1'] = []


else:
    with open('../Data/best_results/pos_adj.pickle', 'rb') as f:
        best_res = pickle.load(f)

In [43]:
for i in range(len(models)):
    best_res[labels[i]]['best_dim_set'].append(gains_df.idxmax()[i])
    best_res[labels[i]]['best_dims'].append(dims[labels[i]][gains_df.idxmax()[i]])
    best_res[labels[i]]['accs'].append(accs_df.loc[accs_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['gains'].append(gains_df.loc[gains_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['medians_0'].append(medians[labels[i]][accs_df.idxmax()[i]]['0'])
    best_res[labels[i]]['medians_1'].append(medians[labels[i]][accs_df.idxmax()[i]]['1'])

In [44]:
with open('../Data/best_results/pos_adj.pickle', 'wb') as f:
    pickle.dump(best_res, f)

# Conclusion